## 数独问题

* 林俊禧
* 2022205222

**任务简介**
数独（Sudoku）是源自18世纪瑞士的一种数学游戏。是一种运用纸、笔进行演算的逻辑游戏。玩家需要根据9×9盘面上的已知数字，推理出所有剩余空格的数字，并满足每一行、每一列、每一个粗线宫（3*3）内的数字均含1-9，不重复。

数独有空间特征，因为它有特殊的数字排列，而CNN擅长提取空间特征，可尝试使用CNN来求解数独。

**数据简介**
数据来源：https://www.kaggle.com/bryanpark/sudoku

数据集包含2列。`quizzes`栏目是未解的游戏，`solutions`栏目各自的已解游戏。每场比赛都由81个数字组成的字符串表示。

**要求**
1. 分析数独游戏的特点，设计实现的方法。
2. 在Kaggle网站上，下载数据。
3. 编写神经网络代码，测试自己算法的效果如何。
4. 测试所研究方法的效果。
5. 分析自己实现的方法的问题，以及如何改进。
6. 深入思考，如何使用强化学习的方法实现求解数独问题？

**游戏特点**
数独是一种流行的数字谜题，它要求玩家在9X9网格中填入数字，以便每一列、每一行和9个子网格中的每一个都包含从1到9的所有数字。在解决数独谜题时，玩家需要同时依赖网格上数字的位置和关系来作出填充的判断。受这些语义的启发，可以尝试使用CNN来解决数独难题。

In [53]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [54]:
n_epochs = 2
batch_size = 64
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [55]:
class DatasetInstance(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self,index):
        sudoku = self.X[index]
        target = self.y[index]
        return sudoku,target

In [56]:
def norm(data):
    return (data/9)-0.5

def get_data(file):
    data = pd.read_csv(file)
    quizzes_raw = data['quizzes']
    solutions_raw = data['solutions']

    quizzes = []
    solutions = []

    print("Loading Sudokus")
    for i in tqdm(quizzes_raw):
        x = np.array([int(j) for j in i]).reshape((1,9,9))
        quizzes.append(x)
    quizzes = norm(np.array(quizzes))

    print("Loading Solutions")
    for i in tqdm(solutions_raw):
        x = np.array([int(j) for j in i]).reshape((9,9)) - 1
        solutions.append(x)

    solutions = np.array(solutions)

    del quizzes_raw
    del solutions_raw

    X_train, X_test, y_train, y_test = train_test_split(quizzes, solutions, test_size=0.2, random_state=42)
    X_train = torch.tensor(X_train).type(torch.FloatTensor)
    X_test = torch.tensor(X_test).type(torch.FloatTensor)
    y_train = torch.tensor(y_train).type(torch.LongTensor)
    y_test = torch.tensor(y_test).type(torch.LongTensor)

    train_dataset = DatasetInstance(X_train, y_train)
    test_dataset = DatasetInstance(X_test, y_test)

    return train_dataset, test_dataset

In [57]:
train_dataset, test_dataset = get_data('sudoku.csv')
#convert to tensors

# load training data in batches
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

test_loader = DataLoader(test_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=0)

Loading Sudokus


100%|██████████| 1000000/1000000 [00:10<00:00, 94349.51it/s]


Loading Solutions


100%|██████████| 1000000/1000000 [00:10<00:00, 95745.38it/s]


In [58]:
class Conv2dLayer(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, bias=True, padding_layer=torch.nn.ReflectionPad2d):
        """It only support square kernels and stride=1, dilation=1, groups=1."""
        super(Conv2dLayer, self).__init__()
        ka = kernel_size // 2
        kb = ka - 1 if kernel_size % 2 == 0 else ka
        self.net = nn.Sequential(
            padding_layer((ka,kb,ka,kb)),
            nn.Conv2d(in_channels, out_channels, kernel_size, bias=bias),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(True)
        )
    def forward(self, x):
        return self.net(x)

class SudokuCNN(nn.Module):
    def __init__(self):
        super(SudokuCNN, self).__init__()
        self.conv_layers = nn.Sequential(Conv2dLayer(1,512,3), #1
                                         Conv2dLayer(512,512,3),#2
                                         Conv2dLayer(512,512,3),#3
                                         Conv2dLayer(512,512,3),#4
                                         Conv2dLayer(512,512,3),#5
                                         Conv2dLayer(512,512,3),#6
                                         Conv2dLayer(512,512,3),#7
                                         Conv2dLayer(512,512,3),#8
                                         Conv2dLayer(512,512,3),#9
                                         Conv2dLayer(512,512,3),#10
                                         Conv2dLayer(512,512,3),#11
                                         Conv2dLayer(512,512,3),#12
                                         Conv2dLayer(512,512,3),#13
                                         Conv2dLayer(512,512,3),#14
                                         Conv2dLayer(512,512,3))#15
        self.last_conv = nn.Conv2d(512, 9, 1)

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.last_conv(x)
        return x

In [59]:
net = SudokuCNN()
net.to(device)

criterion = nn.CrossEntropyLoss().cuda()
optimizer =torch.optim.Adam(net.parameters(), lr=1e-4)
min_loss = float('inf')

In [60]:
def train_net(epoch, n_epochs, net, criterion, optimizer, device, train_loader):

    # prepare the net for training
    net.train()

    print(device)
    running_loss = 0.0

    # train on batches of data, assumes you already have train_loader
    for batch_i, data in enumerate(train_loader):
        # get the input  and their corresponding targets
        sudoku = data[0]
        target = data[1]
        # zero the parameter (weight) gradients
        optimizer.zero_grad()

        sudoku = sudoku.to(device)
        target = target.to(device)
        sudoku = Variable(sudoku)
        target = Variable(target)
        # forward pass to get outputs
        output = net(sudoku)

        # calculate the loss between predicted and target
        loss = criterion(output , target)

        # backward pass to calculate the weight gradients
        loss.backward()

        # update the weights
        optimizer.step()

        # print loss statistics
        running_loss += loss.item()
        if batch_i % 200 == 0:    # print every 10 batches
            print('Epoch: {}, Batch: {}, Avg. Loss: {}'.format(epoch + 1, batch_i+1, running_loss/10))
            running_loss = 0.0

    #print('Finished Training')

In [61]:
def val_net(epoch, n_epochs, net, criterion, optimizer, device, test_loader):
    net.eval()
    val_loss=0.0
    print(device)
    correct = 0
    ndata = 0
    with torch.set_grad_enabled(False):
        for i,data in enumerate(test_loader):
            sudoku = data[0]
            target = data[1]
            optimizer.zero_grad()
            sudoku = sudoku.to(device)
            target = target.to(device)
            # forward pass to get outputs
            output = net(sudoku)

            # calculate the loss between predicted and target
            loss = criterion(output , target)

            val_loss+=loss.data.item()
            preds = torch.argmax(output,dim=1)

            for j in range(len(preds)):
                if (preds[j] == target[j]).all():
                    correct += 1
            ndata += preds.shape[0]
            if i==0 or i % 1000 == 0:
                print(f'Test Loss at {i}: {val_loss / (i + 1)} Accuracy:{correct}/{ndata}')
    print(f'Test Loss at {i}: {val_loss / (i + 1)} Accuracy:{correct/ndata}')
    return val_loss / (i + 1)

In [ ]:
for epoch in range(n_epochs):
    print("=====Training=======")
    train_net(epoch, n_epochs, net, criterion, optimizer, device, train_loader)
    print("=====Validation=======")
    loss = val_net(epoch, n_epochs, net, criterion, optimizer, device, test_loader)
    if loss < min_loss:
        print("=====Saving=======")
        model_dir = './saved_models/'
        name = 'SudokuCNN.pt'
        min_loss = loss
        torch.save(net.state_dict(), model_dir+name)

=====Training=======
cpu
Epoch: 1, Batch: 1, Avg. Loss: 0.2247058391571045


In [ ]:
def denorm(a):

    return (a+.5)*9

def inference_sudoku(sample,model):

    '''
        This function solve the sudoku by filling blank positions one by one.
    '''

    feat = sample.clone().cuda()

    while True:

        out = model(feat.reshape((1,1,9,9)))

        pred = torch.argmax(out, axis=1).reshape((9,9)) + 1
        prob,_ = torch.max(out, axis=1)
        #prob = prob.reshape((9,9))
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)

        if(mask.sum()==0):
            break
        prob = torch.sigmoid(prob)
        prob_new = prob.flatten() * mask.flatten()

        ind = torch.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    return pred

In [ ]:


def test_accuracy(epoch, n_epochs, net, device, test_loader):
    net.eval()
    print(device)
    correct = 0
    ndata = 0
    with torch.set_grad_enabled(False):
        for i,data in enumerate(test_loader):
            sudoku = data[0]
            target = data[1]
            sudoku = sudoku.to(device)
            target = target.to(device)


            for j in range(len(sudoku)):
                if (inference_sudoku(sudoku[j],net) == target[j] + 1).all():
                    correct += 1

            ndata += sudoku.shape[0]
            if ndata>=1000:
                print(f' Accuracy:{correct}/{ndata}')
    print(f'Final Accuracy:{correct/ndata}')

In [ ]:
test_accuracy(0, 0, net, device, test_loader)